In [ ]:
!pip install python-terrier

     |████████████████████████████████| 95 kB 2.5 MB/s 
     |████████████████████████████████| 1.1 MB 39.4 MB/s 
     |████████████████████████████████| 69 kB 6.8 MB/s 
     |████████████████████████████████| 255 kB 47.1 MB/s 
     |████████████████████████████████| 1.8 MB 43.5 MB/s 
     |████████████████████████████████| 596 kB 36.0 MB/s 
     |████████████████████████████████| 72 kB 1.1 MB/s 
     |████████████████████████████████| 6.3 MB 35.2 MB/s 
     |████████████████████████████████| 294 kB 50.5 MB/s 
     |████████████████████████████████| 126 kB 47.9 MB/s 
     |████████████████████████████████| 291 kB 50.4 MB/s 
     |████████████████████████████████| 45 kB 3.0 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.7.1-py3-none-any.whl size=102452 sha256=183805a12129f515c995289fb9a84571ef8454f8d42eec190043b9e54dc1029f
  Stored in directory: /root/.cache/pip/wheels/c0/9a/c8/1c2d9ec6a1494bb54f47e0d2627b5ed7b2de704b66723d3417
  Created wheel for ir-measures: filen

In [ ]:
import pyterrier as pt
import pandas as pd
import os
from pyterrier.measures import *
import re

In [ ]:
foodresults = pd.read_csv("foodresults.csv", encoding="latin-1")
index_result = pd.read_csv("index_result.csv")
df = pd.read_csv("dataset_with_relevance.csv",encoding="latin-1")

In [ ]:
if not pt.started():
  pt.init()

terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.7.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)


In [ ]:
import re
s="sdfgfs$%$#%#$9998 sdfg"
word1 = " ".join(re.findall("[a-zA-Z]+", s))
word1

'sdfgfs sdfg'

In [ ]:
data=[]
for index,row in df.iterrows():
  query = " ".join(re.findall("[a-zA-Z]+", row["query"]))
  recipe_id = row["recipe_id"]
  recipe_name = " ".join(re.findall("[a-zA-Z]+", row["recipe_name"]))
  Relevance = row["Relevance"]
  ingredients = " ".join(re.findall("[a-zA-Z]+", str(row["ingredients"])))
  recipe = " ".join(re.findall("[a-zA-Z]+", row["recipe"]))
  rate = row["rate"]
  views = row["views"]
  description = " ".join(re.findall("[a-zA-Z]+", row["description"]))
  keywords = " ".join(re.findall("[a-zA-Z]+", str(row["keywords"])))
  calories = row["calories"]
  fatContent = row["fatContent"]
  proteinContent = row["proteinContent"]
  data.append([query,recipe_id,recipe_name,Relevance,ingredients,recipe,
               rate,views,description,keywords,calories,fatContent,proteinContent])
new_df = pd.DataFrame(data,columns=["query","docno","recipe_name","Relevance","ingredients","recipe",
               "rate","views","description","keywords","calories","fatContent","proteinContent"])

In [ ]:
new_df.head(6)

,query,docno,recipe_name,Relevance,ingredients,recipe,rate,views,description,keywords,calories,fatContent,proteinContent
0,tomato,6492,Macaroni and Tomatoes,5,large potatoes teaspoon salt eggs lightly beat...,Scrub potatoes well but do not peel In a sauce...,4.5,6,Make and share this Potato Plum Dumplings reci...,Vegetable Hungarian European Low Protein Healt...,270.6,8.9,5.8
1,tomato,50060,Tofu Stuffed Cherry Tomatoes,5,onion diced garlic cloves minced carrot diced ...,Sweat onion and garlic in small amount of oil ...,0.0,0,Creamy fat free low cal and a serving of veggi...,Lactose Free Low Protein Low Cholesterol Egg F...,107.6,0.8,3.8
2,tomato,54375,Canned Tomatoes Vine Ripened Tomatoes in Tomat...,5,butter recipe cake mix baked in a greased x cm...,For best results bake the cake the day before ...,0.0,0,Make and share this Lamingtons recipe from Foo...,Low Protein Low Cholesterol Healthy Kid Friend...,179.2,3.1,1.3
3,tomato,11958,Chilli Tomato Sauce Tomato Salsa,5,large mushrooms cup breadcrumbs cup parmesan c...,clean mushrooms with damp cloth remove stems a...,5.0,1,Make and share this Moms Stuffed Mushrooms rec...,Healthy Mins For Large Groups,16.8,0.7,1.4
4,tomato,51985,Cherry Tomato Planets,5,teaspoons sesame oil tablespoons gingerroot sh...,Place the sesame oil in a hot wok with the gin...,0.0,0,This is from the Donna Hay cookbook quot Cooki...,Asian Low Protein Low Cholesterol Healthy Mins...,86.8,2.3,0.5
5,tomato,16912,BLANCHING TOMATOES for COOKING amp CANNING,4,ounces large shrimp peeled and deveined tables...,Heat the oil in a large heavy bottomed skillet...,0.0,0,Make and share this Cilantro Lime Shrimp recip...,Very Low Carbs Healthy Mins,165.6,5.4,24.6


In [ ]:

data=[]
foodresults["docno"] = range(0,len(foodresults))
foodresults["recipe_id"] = range(0,len(foodresults))
foodresults.head(5)
for index,row in foodresults.iterrows():
  docno = "d"+str(row["docno"])
  recipe_id = str(row["recipe_id"])
  recipe_name = " ".join(re.findall("[a-zA-Z]+", row["name"]))
  ingredients = " ".join(re.findall("[a-zA-Z]+", str(row["ingredients"])))
  recipe = " ".join(re.findall("[a-zA-Z]+", str(row["recipe"])))
  rate = str(int(row["rate"]))
  views = str(int(row["views"]))
  description = " ".join(re.findall("[a-zA-Z]+", row["description"]))
  keywords = " ".join(re.findall("[a-zA-Z]+", str(row["keywords"])))
  calories = row["calories"]
  fatContent = row["fatContent"]
  proteinContent = row["proteinContent"]
  data.append([recipe_id,recipe_name,ingredients,recipe,
               rate,views,description,keywords,calories,fatContent,proteinContent])
whole_df = pd.DataFrame(data,columns=["docno","recipe_name","ingredients","recipe",
               "rate","views","description","keywords","calories","fatContent","proteinContent"])

In [ ]:
whole_df.head(5)

,docno,recipe_name,ingredients,recipe,rate,views,description,keywords,calories,fatContent,proteinContent
0,0,Chinese Fried Rice,cup finely chopped onion tablespoons oil egg l...,Heat tbsp oil in wok add chopped onions and st...,5,541,This Chinese fried rice has the flavor those o...,Pork Poultry Rice Meat Chinese Asian Low Chole...,497.8,16.3,22.3
1,1,Perfect Pork Tenderloin,lb boneless pork tenderloin salt and pepper dr...,NOTE nbsp The success of this cooking method w...,5,434,My way to cook a pork tenderloin or a boneless...,Meat Very Low Carbs High Protein Healthy High ...,136.2,4.0,23.4
2,2,Pulled Pork Crock Pot,lbs pork roast shoulder or butt large onions c...,Slice one onion and place in crock pot Put in ...,5,642,I found this pulled pork recipe years ago and ...,Meat Healthy Beginner Cook Easy,335.1,6.9,40.2
3,3,The Most Wonderful Gingerbread Cookies,cups all purpose flour teaspoons baking powder...,In a small bowl whisk together flour baking po...,4,559,This is my very favorite gingerbread cookie re...,Cookie Brownie Healthy Christmas Oven Hours,134.2,3.2,1.9
4,4,Restaurant Teriyaki Sauce,cup soy sauce cup water teaspoon ground ginger...,Mix all but cornstarch and c water in a sauce ...,5,341,You know the nice sweet thick Teriyaki sauce y...,Low Protein Low Cholesterol Healthy Mins Easy,290.5,0.1,5.3


In [ ]:
whole_index_dir = "./whole_docs_index"
whole_indexer = pt.DFIndexer(whole_index_dir, overwrite=True)
whole_index_ref = whole_indexer.index(whole_df["recipe"], whole_df["docno"],
                    whole_df["recipe_name"], whole_df["keywords"], 
                    whole_df["description"], whole_df["ingredients"],
                    whole_df["rate"],whole_df["views"])
whole_index_ref.toString()

'./whole_docs_index/data.properties'

In [ ]:
whole_index = pt.IndexFactory.of(whole_index_ref)

In [ ]:
topics = pd.read_csv("topics.csv")
qrels = pd.read_csv("qrel.csv")
qrels = qrels.drop(columns=["iteration"])

In [ ]:
topics.head(5)

,qid,query
0,1,apple
1,2,bacon
2,3,baking soda
3,4,barbecue ribs
4,5,beef


In [ ]:
topics["qid"] = topics["qid"].astype(str)

In [ ]:
topics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   qid     30 non-null     object
 1   query   30 non-null     object
dtypes: object(2)
memory usage: 608.0+ bytes


In [ ]:
qrels["qid"] = qrels["qid"].astype(str)
qrels["docno"] = qrels["docno"].astype(str)

**Machine Learning**

In [ ]:
# constants
RANK_CUTOFF = 100


In [83]:
bm25 = pt.BatchRetrieve(whole_index, wmodel="BM25")
tfidf = pt.BatchRetrieve(whole_index, wmodel="TF_IDF")
sdm = pt.rewrite.SDM()
qe = pt.rewrite.Bo1QueryExpansion(whole_index)
query_expansion = bm25>>qe>>bm25
ltr_feats_new1 = (bm25 % RANK_CUTOFF) >> pt.text.get_text(whole_index, ["recipe_name","keywords","description","ingredients","rate","views"]) >> (
    pt.transformer.IdentityTransformer()
    ** 
    (pt.text.scorer(body_attr="recipe_name", takes='docs', wmodel='BM25'))
    **
    (pt.text.scorer(body_attr="keywords", takes='docs', wmodel='BM25'))
    ** 
    (pt.text.scorer(body_attr="description", takes='docs', wmodel='BM25'))
    ** 
    (pt.text.scorer(body_attr="ingredients", takes='docs', wmodel='BM25'))
    **
    pt.apply.doc_score(lambda row: int(row["rate"]))
    **
    pt.apply.doc_score(lambda row: int(row["views"]))
    **
    query_expansion
)
# for reference, lets record the feature names here too
fnames=["f1","f2"]

In [84]:
SEED=42

from sklearn.model_selection import train_test_split

tr_va_topics, test_topics = train_test_split(topics, test_size=0.15, random_state=SEED)
train_topics, valid_topics =  train_test_split(tr_va_topics, test_size=0.17, random_state=SEED)

In [ ]:
len(train_topics)

20

In [ ]:
len(test_topics)

5

In [ ]:
len(valid_topics)

5

In [85]:
from sklearn.ensemble import RandomForestRegressor
pipeline = pt.FeaturesBatchRetrieve(whole_index, wmodel="BM25", features=["WMODEL:Tf"])
rf = RandomForestRegressor(n_estimators=400, verbose=1, random_state=42, n_jobs=2)
rf_pipe = ltr_feats_new1 >> pt.ltr.apply_learned_model(rf)
%time rf_pipe.fit(train_topics, qrels)


19:52:29.967 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
19:52:31.328 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
19:52:32.862 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
19:52:34.297 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Def

/usr/local/lib/python3.7/dist-packages/pyterrier/transformer.py:760: UserWarning: Got number of results different expected from ComposedPipeline(ComposedPipeline(BR(/content/whole_docs_index/data.properties,{'terrierql': 'on', 'parsecontrols': 'on', 'parseql': 'on', 'applypipeline': 'on', 'localmatching': 'on', 'filters': 'on', 'decorate': 'on', 'wmodel': 'BM25'},{'querying.processes': 'terrierql:TerrierQLParser,parsecontrols:TerrierQLToControls,parseql:TerrierQLToMatchingQueryTerms,matchopql:MatchingOpQLParser,applypipeline:ApplyTermPipeline,context_wmodel:org.terrier.python.WmodelFromContextProcess,localmatching:LocalManager$ApplyLocalMatching,qe:QueryExpansion,labels:org.terrier.learning.LabelDecorator,filters:LocalManager$PostFilterProcess,decorate:SimpleDecorateProcess', 'querying.postfilters': 'decorate:SimpleDecorate,site:SiteFilter,scope:Scope', 'querying.default.controls': 'wmodel:DPH,parsecontrols:on,parseql:on,applypipeline:on,terrierql:on,localmatching:on,filters:on,decorat

In [86]:
import lightgbm as lgb

# this configures LightGBM as LambdaMART
lmart_l = lgb.LGBMRanker(
    task="train",
    silent=False,
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=1,
    max_bin=255,
    num_leaves=31,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[10],
    ndcg_at=[10],
    eval_at=[10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=100,
    early_stopping_rounds=5
)

lmart_x_pipe = ltr_feats_new1 >> pt.ltr.apply_learned_model(lmart_l, form="ltr", fit_kwargs={'eval_at':[10]})

%time lmart_x_pipe.fit(train_topics, qrels, valid_topics, qrels)

19:52:53.795 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
19:52:55.609 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
19:52:57.158 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
19:52:58.638 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Def

/usr/local/lib/python3.7/dist-packages/pyterrier/transformer.py:760: UserWarning: Got number of results different expected from ComposedPipeline(ComposedPipeline(BR(/content/whole_docs_index/data.properties,{'terrierql': 'on', 'parsecontrols': 'on', 'parseql': 'on', 'applypipeline': 'on', 'localmatching': 'on', 'filters': 'on', 'decorate': 'on', 'wmodel': 'BM25'},{'querying.processes': 'terrierql:TerrierQLParser,parsecontrols:TerrierQLToControls,parseql:TerrierQLToMatchingQueryTerms,matchopql:MatchingOpQLParser,applypipeline:ApplyTermPipeline,context_wmodel:org.terrier.python.WmodelFromContextProcess,localmatching:LocalManager$ApplyLocalMatching,qe:QueryExpansion,labels:org.terrier.learning.LabelDecorator,filters:LocalManager$PostFilterProcess,decorate:SimpleDecorateProcess', 'querying.postfilters': 'decorate:SimpleDecorate,site:SiteFilter,scope:Scope', 'querying.default.controls': 'wmodel:DPH,parsecontrols:on,parseql:on,applypipeline:on,terrierql:on,localmatching:on,filters:on,decorat

19:53:05.000 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
19:53:05.427 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
19:53:05.894 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
19:53:06.336 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Def

/usr/local/lib/python3.7/dist-packages/pyterrier/transformer.py:760: UserWarning: Got number of results different expected from ComposedPipeline(ComposedPipeline(BR(/content/whole_docs_index/data.properties,{'terrierql': 'on', 'parsecontrols': 'on', 'parseql': 'on', 'applypipeline': 'on', 'localmatching': 'on', 'filters': 'on', 'decorate': 'on', 'wmodel': 'BM25'},{'querying.processes': 'terrierql:TerrierQLParser,parsecontrols:TerrierQLToControls,parseql:TerrierQLToMatchingQueryTerms,matchopql:MatchingOpQLParser,applypipeline:ApplyTermPipeline,context_wmodel:org.terrier.python.WmodelFromContextProcess,localmatching:LocalManager$ApplyLocalMatching,qe:QueryExpansion,labels:org.terrier.learning.LabelDecorator,filters:LocalManager$PostFilterProcess,decorate:SimpleDecorateProcess', 'querying.postfilters': 'decorate:SimpleDecorate,site:SiteFilter,scope:Scope', 'querying.default.controls': 'wmodel:DPH,parsecontrols:on,parseql:on,applypipeline:on,terrierql:on,localmatching:on,filters:on,decorat

[1]	valid_0's ndcg@10: 0.84837
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's ndcg@10: 0.847061
[3]	valid_0's ndcg@10: 0.852875
[4]	valid_0's ndcg@10: 0.843621
[5]	valid_0's ndcg@10: 0.824009
[6]	valid_0's ndcg@10: 0.830848
[7]	valid_0's ndcg@10: 0.838741
[8]	valid_0's ndcg@10: 0.850101
Early stopping, best iteration is:
[3]	valid_0's ndcg@10: 0.852875
CPU times: user 27.5 s, sys: 522 ms, total: 28 s
Wall time: 18.6 s


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [87]:
feature_names = ["bm25","recipe name","keywords","description","ingredients","rate","views","query expansion"]
print("*************** this is rf ***************")
for i in range(len(feature_names)):
  print(f"feature: {feature_names[i]}, importance: {rf.feature_importances_[i]}")
print("*************** this is lgbm ***************")
for i in range(len(feature_names)):
  print(f"feature: {feature_names[i]}, importance: {lmart_l.feature_importances_[i]}")

*************** this is rf ***************
feature: bm25, importance: 0.17660586175499862
feature: recipe name, importance: 0.2857594413452853
feature: keywords, importance: 0.01537880784511263
feature: description, importance: 0.04399860649627774
feature: ingredients, importance: 0.2200536029600199
feature: rate, importance: 0.014028855601104307
feature: views, importance: 0.03040160542210648
feature: query expansion, importance: 0.21377321857509501
*************** this is lgbm ***************
feature: bm25, importance: 625.0594865083694
feature: recipe name, importance: 2218.44108915329
feature: keywords, importance: 103.33729934692383
feature: description, importance: 324.30929946899414
feature: ingredients, importance: 1587.351601243019
feature: rate, importance: 62.7318000793457
feature: views, importance: 115.9630491733551
feature: query expansion, importance: 1522.1083855628967


In [89]:
pt.Experiment(
    [rf_pipe % 50,lmart_x_pipe, bm25 % 50, tfidf % 50],
    test_topics,
    qrels,
    eval_metrics=["map", "ndcg", nDCG@10, "mrt"],
    names=["rf model","lgbm model","bm25","tfidf"],
    baseline = 3
)

19:54:50.086 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
19:54:50.551 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
19:54:51.074 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
19:54:51.578 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Def

/usr/local/lib/python3.7/dist-packages/pyterrier/transformer.py:760: UserWarning: Got number of results different expected from ComposedPipeline(ComposedPipeline(BR(/content/whole_docs_index/data.properties,{'terrierql': 'on', 'parsecontrols': 'on', 'parseql': 'on', 'applypipeline': 'on', 'localmatching': 'on', 'filters': 'on', 'decorate': 'on', 'wmodel': 'BM25'},{'querying.processes': 'terrierql:TerrierQLParser,parsecontrols:TerrierQLToControls,parseql:TerrierQLToMatchingQueryTerms,matchopql:MatchingOpQLParser,applypipeline:ApplyTermPipeline,context_wmodel:org.terrier.python.WmodelFromContextProcess,localmatching:LocalManager$ApplyLocalMatching,qe:QueryExpansion,labels:org.terrier.learning.LabelDecorator,filters:LocalManager$PostFilterProcess,decorate:SimpleDecorateProcess', 'querying.postfilters': 'decorate:SimpleDecorate,site:SiteFilter,scope:Scope', 'querying.default.controls': 'wmodel:DPH,parsecontrols:on,parseql:on,applypipeline:on,terrierql:on,localmatching:on,filters:on,decorat

19:54:54.986 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
19:54:55.441 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
19:54:55.958 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
19:54:56.458 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Def

/usr/local/lib/python3.7/dist-packages/pyterrier/transformer.py:760: UserWarning: Got number of results different expected from ComposedPipeline(ComposedPipeline(BR(/content/whole_docs_index/data.properties,{'terrierql': 'on', 'parsecontrols': 'on', 'parseql': 'on', 'applypipeline': 'on', 'localmatching': 'on', 'filters': 'on', 'decorate': 'on', 'wmodel': 'BM25'},{'querying.processes': 'terrierql:TerrierQLParser,parsecontrols:TerrierQLToControls,parseql:TerrierQLToMatchingQueryTerms,matchopql:MatchingOpQLParser,applypipeline:ApplyTermPipeline,context_wmodel:org.terrier.python.WmodelFromContextProcess,localmatching:LocalManager$ApplyLocalMatching,qe:QueryExpansion,labels:org.terrier.learning.LabelDecorator,filters:LocalManager$PostFilterProcess,decorate:SimpleDecorateProcess', 'querying.postfilters': 'decorate:SimpleDecorate,site:SiteFilter,scope:Scope', 'querying.default.controls': 'wmodel:DPH,parsecontrols:on,parseql:on,applypipeline:on,terrierql:on,localmatching:on,filters:on,decorat

,name,map,ndcg,nDCG@10,mrt,map +,map -,map p-value,ndcg +,ndcg -,ndcg p-value,nDCG@10 +,nDCG@10 -,nDCG@10 p-value
0,rf model,0.315263,0.471756,0.660591,987.669627,3.0,2.0,0.87680,3.0,2.0,0.622340,0.0,5.0,0.049902
1,lgbm model,0.463418,0.615982,0.778975,879.488496,5.0,0.0,0.01838,5.0,0.0,0.002822,3.0,2.0,0.785933
2,bm25,0.322835,0.459698,0.762654,129.161880,2.0,1.0,0.91607,3.0,0.0,0.343074,1.0,1.0,0.569462
3,tfidf,0.322784,0.456783,0.764173,155.015278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
